In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import re
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [2]:
URL = "https://www.travelandleisure.com/attractions/worlds-most-visited-sacred-sites"

In [3]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geolocator = Nominatim(user_agent="geocode_religious_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)

In [4]:
webpage = requests.get(URL, headers=HEADER)
webpage

<Response [200]>

In [5]:
soup = BeautifulSoup(webpage.content, "html.parser")
#soup

In [6]:
titles = soup.find_all("span", attrs = {'class': 'mntl-sc-block-heading__text'})
data = soup.find_all("p", attrs = {'class': 'comp mntl-sc-block mntl-sc-block-html'})
#print(data[53].text.strip("\n").split(":")[1].strip().split(" ")[0].split("–")[0])
#len(data)
temp_title = titles[1].text.strip()
title = re.sub(r"No\. [0-9]\s", "", temp_title)
title

'Kashi Vishwanath Temple, Varanasi, India'

In [7]:
for i in range(20):
    temp_title = titles[i].text.strip()
    title = re.sub(r"No\. \b([1-9]|[12][0-9])\b\s", "", temp_title)
    #print(str(i) + "     " + title)
    titles[i] = title


In [8]:
visitors_list = []
sample_word = "Annual Visitors:"
for i in range(len(data)):
    if sample_word in data[i].text:
        #print(i)
        visitor = data[i].text.strip("\n").split(":")[1].strip().split(" ")[0].split("–")[0]
        #print(str(i) + "  " + visitor)
        visitors_list.append(visitor)
        if len(visitors_list) == 20:
            break
visitors_list

['30',
 '21.9',
 '20',
 '18.25',
 '13.65',
 '10.5',
 '10',
 '10',
 '10',
 '8.5',
 '7',
 '6.6',
 '6',
 '6',
 '6',
 '5',
 '5.5',
 '5.5',
 '5',
 '5']

In [9]:
final_title_list = titles[:21] + visitors_list
#for i in range(len(final_title_list)):
    #print(str(i)+ "  " + final_title_list[i])
print(len(final_title_list))
print(final_title_list[0])
print(final_title_list[1])

41
Meiji Shrine and Sensoji-Temple, Tokyo
Kashi Vishwanath Temple, Varanasi, India


In [10]:
city_address = []
for i in range(int(len(final_title_list)/2)):

    # GETTING RAW DATA
    try:
        location_description = geocoder.geocode(final_title_list[i])
    except Exception:
        print(str(i) + " " + "No data found for location: " + final_title_list[i])
        continue

    # GETTING ADDRESS VALUES
    loc_title = final_title_list[i].split(",")[0]
    latitude = str(location_description[0]['geometry']['lat'])
    longitude = str(location_description[0]['geometry']['lng'])
    try:
        city = location_description[0]['components']['city']
    except Exception:
        try:
            city = location_description[0]['components']['town']
        except Exception:
            try:
                city = location_description[0]['components']['state']
            except Exception:
                city = "NULL"
    country = location_description[0]['components']['country']
    country_code = location_description[0]['components']['country_code']
    continent = location_description[0]['components']['continent']
    visitors = final_title_list[i+21]
    
    #GETTING POSTCODE
    lat_long = str(latitude) + ","+ str(longitude) 
    location2 = geolocator.reverse(lat_long, language="en")
    try:
        postcode = location2.raw['address']['postcode'].replace(" ","")
    except:
        postcode = "NULL"
        
    # ADDING VARIABLES TO DICTIONARY
    address_dict = {'Location': loc_title, 'City/State': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode, 'Visitors (in Millions)': visitors}
    city_address.append(address_dict)
    
print(city_address)

[{'Location': 'Meiji Shrine and Sensoji-Temple', 'City/State': 'Tokyo', 'Country': 'Japan', 'Country Code': 'JP', 'Continent': 'Asia', 'Latitude': '35.69', 'Longitude': '139.69', 'Postcode': '163-8001', 'Visitors (in Millions)': '30'}, {'Location': 'Kashi Vishwanath Temple', 'City/State': 'Varanasi', 'Country': 'India', 'Country Code': 'IN', 'Continent': 'Asia', 'Latitude': '25.32', 'Longitude': '83.01', 'Postcode': '221001', 'Visitors (in Millions)': '21.9'}, {'Location': 'Basilica of Our Lady of Guadalupe', 'City/State': 'Mexico City', 'Country': 'Mexico', 'Country Code': 'MX', 'Continent': 'North America', 'Latitude': '19.48', 'Longitude': '-99.12', 'Postcode': '07050', 'Visitors (in Millions)': '20'}, {'Location': 'Tirupati Tirumala Devasthanams Temple', 'City/State': 'NULL', 'Country': 'India', 'Country Code': 'IN', 'Continent': 'Asia', 'Latitude': '22.0', 'Longitude': '79.0', 'Postcode': '480001', 'Visitors (in Millions)': '18.25'}, {'Location': 'Notre Dame Cathedral', 'City/Stat

In [11]:
ca_most_religious_df = pd.DataFrame.from_dict(city_address)
ca_most_religious_df

,Location,City/State,Country,Country Code,Continent,Latitude,Longitude,Postcode,Visitors (in Millions)
0,Meiji Shrine and Sensoji-Temple,Tokyo,Japan,JP,Asia,35.69,139.69,163-8001,30
1,Kashi Vishwanath Temple,Varanasi,India,IN,Asia,25.32,83.01,221001,21.9
2,Basilica of Our Lady of Guadalupe,Mexico City,Mexico,MX,North America,19.48,-99.12,07050,20
3,Tirupati Tirumala Devasthanams Temple,NULL,India,IN,Asia,22.0,79.0,480001,18.25
4,Notre Dame Cathedral,Ho Chi Minh City Saigon,Vietnam,VN,Asia,10.78,106.7,71006,13.65
5,Sacré Coeur Basilica,Paris,France,FR,Europe,48.89,2.34,75018,10.5
6,Naritasan Shinshoji Temple,Narita,Japan,JP,Asia,35.79,140.32,286-0024,10
7,Tsurugaoka Hachimangu Shrine,Kamakura,Japan,JP,Asia,35.33,139.56,248-8588,10
8,Kiyomizu-dera and Kinkaku-ji Temples,Kyoto,Japan,JP,Asia,35.02,135.75,602-8570,10
9,Ise Jingu,Ise,Japan,JP,Asia,34.46,136.72,516-0023,8.5
